# Importing necessary libraries

In [1]:
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidi

In [ ]:
from datasets import load_dataset

from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                         DataCollatorWithPadding,TrainingArguments,Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd 

2025-08-29 15:37:28.866128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756481849.206434      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756481849.300037      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Loading the Dataset 

In [4]:
dataset_dict = load_dataset("RohitWani17/legalcompliance")

README.md:   0%|          | 0.00/507 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/34.7k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/98 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21 [00:00<?, ? examples/s]

In [5]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 98
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 21
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 21
    })
})

# Loading the model

In [6]:
# Load model directly
model_path = "openai-community/gpt2"

# Load model toknizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load model with mutlti-class classification head
id2label = {0: 'CAN-SPAM Compliant', 1: 'CAN-SPAM Violation', 2: 'CCPA Disclosure', 3: 'Non-Disclosure'}
label2id = {'CAN-SPAM Compliant': 0, 'CAN-SPAM Violation': 1, 'CCPA Disclosure': 2, 'Non-Disclosure': 3}
model = AutoModelForSequenceClassification.from_pretrained(model_path, 
                                                           num_labels=4, 
                                                           id2label=id2label, 
                                                           label2id=label2id,)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Preprocessing data 

In [7]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, add_prefix_space=True)

In [8]:
# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# No need to resize embeddings if using an existing token

In [9]:
model.config.pad_token_id = tokenizer.pad_token_id

In [10]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [11]:
# tokenize datasets
tokenized_dataset = dataset_dict.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 98
    })
    validation: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 21
    })
    test: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 21
    })
})

In [12]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Creating the evaluation metrics functions

In [13]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [14]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

# Setting up LORA Configurations

In [15]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules=["c_attn", "c_proj"],)  # Correct modules for GPT-2)

In [16]:
peft_config

LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=4, target_modules={'c_proj', 'c_attn'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [17]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 408,576 || all params: 124,851,456 || trainable%: 0.3272


# Setting up the training arguments

In [18]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 5

In [19]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_path + "-lora-legal_compliance-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    label_names=["labels"],
    report_to="none",
)

In [20]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    processing_class=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.170200,0.467177,{'accuracy': 0.8095238095238095}
2,0.596500,0.585400,{'accuracy': 0.7142857142857143}
3,0.142100,0.078787,{'accuracy': 1.0}
4,0.040300,0.501229,{'accuracy': 0.8571428571428571}
5,0.010500,0.116308,{'accuracy': 0.9523809523809523}


TrainOutput(global_step=65, training_loss=0.39194009464520674, metrics={'train_runtime': 28.6271, 'train_samples_per_second': 17.117, 'train_steps_per_second': 2.271, 'total_flos': 88348249608192.0, 'train_loss': 0.39194009464520674, 'epoch': 5.0})

In [22]:
model.to('cuda')

test_predictions = {"Text":[],"Label":[]}

print("Trained model predictions:")
print("--------------------------")
for i in dataset_dict["test"]:
    inputs = tokenizer.encode(i["text"], return_tensors="pt").to("cuda") 

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices
    print(i["text"])
    print("---------------------------------------------------------------\n\n")
    print("Prediction: ",id2label[predictions.tolist()[0]])
    print("---------------------------------------------------------------\n\n")
    test_predictions["Text"].append(i["text"])    
    test_predictions["Label"].append(id2label[predictions.tolist()[0]])  

Trained model predictions:
--------------------------
Subject: Home Depot - Your online order is ready for pickup

Order Ready for Pickup:

• Drill Set - Dewalt 20V Max ($129.99)
• Wood Screws - 1lb box ($8.99)
• Paint Rollers - 4-pack ($12.49)

Pickup at Customer Service Desk, Store #1234

Home Depot Inc.
2455 Paces Ferry Road
Atlanta, GA 30339

Order confirmations required for purchases. Weekly ads: homedepot.com/unsubscribe
---------------------------------------------------------------


Prediction:  CAN-SPAM Compliant
---------------------------------------------------------------


CLIENT CONFIDENTIALITY POLICY

We collect financial information to provide accounting and tax services.

Client information is protected by professional confidentiality requirements.

Financial data may be shared with tax authorities and regulatory agencies as required.

Clients can access their records and tax documents through our client portal.

Tax & Accounting Professionals
Trusted Financial Servi

In [23]:
test_predictions_df = pd.DataFrame(test_predictions)

In [24]:
test_predictions_df.to_csv("Test_Predictions.csv")

# Saving the model 

In [25]:
save_dir = "legal_compliance_fine_tuned_model"

trainer.save_model(save_dir)        # saves model
trainer.tokenizer.save_pretrained(save_dir)  # saves tokenizer

print(f" Fine-Tuned model saved to {save_dir}")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


 Fine-Tuned model saved to legal_compliance_fine_tuned_model
